# PyDipAPI - Content Parser Tutorial

Dieses Notebook demonstriert die Content-Parser Features von PyDipAPI.

## Was Sie lernen werden:
1. **ProtocolParser** - Plenarprotokolle analysieren
2. **DocumentParser** - Drucksachen verarbeiten
3. **PersonParser** - Abgeordneten-Daten extrahieren
4. **ActivityParser** - Aktivitäten analysieren
5. **Batch-Parsing** - Performance-Optimierung

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage
from pydipapi.parsers import ProtocolParser, DocumentParser, PersonParser, ActivityParser

# API-Schlüssel konfigurieren
API_KEY = "OSOegLs.PR2lwJ1dwCeje9vTj7FPOt3hvpYKtwKkhw"
dip = DipAnfrage(api_key=API_KEY)

print("✅ Content Parser Tutorial bereit!")

✅ Content Parser Tutorial bereit!


## ProtocolParser - Plenarprotokolle analysieren

In [3]:
# Plenarprotokoll laden und parsen
print("📋 Teste ProtocolParser...")
try:
    protocols = dip.get_plenarprotokoll(anzahl=1)
    if protocols:
        parser = ProtocolParser()
        parsed = parser.parse(protocols[0])
        print(f"✅ Protokoll geparst: {len(parsed)} Felder")
        print(f"Verfügbare Felder: {list(parsed.keys())}")
    else:
        print("❌ Keine Protokolle gefunden")
except Exception as e:
    print(f"❌ Fehler: {e}")

📋 Teste ProtocolParser...
✅ Protokoll geparst: 12 Felder
Verfügbare Felder: ['id', 'dokumentart', 'typ', 'vorgangsbezug_anzahl', 'dokumentnummer', 'wahlperiode', 'herausgeber', 'aktualisiert', 'fundstelle', 'titel', 'datum', 'parsed']


## DocumentParser - Drucksachen analysieren

In [4]:
# Drucksachen laden und parsen
print("📄 Teste DocumentParser...")
try:
    documents = dip.get_drucksache(anzahl=3)
    if documents:
        parser = DocumentParser()
        
        # Einzelnes Dokument parsen
        parsed_doc = parser.parse(documents[0])
        print(f"✅ Dokument geparst: {len(parsed_doc)} Felder")
        
        # Batch-Parsing
        batch_results = parser.parse_batch(documents)
        print(f"✅ Batch-Parsing: {len(batch_results)} Dokumente")
    else:
        print("❌ Keine Dokumente gefunden")
except Exception as e:
    print(f"❌ Fehler: {e}")

📄 Teste DocumentParser...
✅ Dokument geparst: 17 Felder
❌ Fehler: 'DocumentParser' object has no attribute 'parse_batch'


## PersonParser - Abgeordnete analysieren

In [5]:
# Personen laden und parsen
print("👥 Teste PersonParser...")
try:
    persons = dip.get_person(anzahl=5)
    if persons:
        parser = PersonParser()
        batch_persons = parser.parse_batch(persons)
        
        print(f"✅ {len(batch_persons)} Personen geparst")
        
        # Beispiel-Person anzeigen
        if batch_persons:
            person = batch_persons[0]
            print(f"Beispiel: {person.get('name', 'Unbekannt')}")
            print(f"Fraktion: {person.get('fraktion', 'Unbekannt')}")
    else:
        print("❌ Keine Personen gefunden")
except Exception as e:
    print(f"❌ Fehler: {e}")

👥 Teste PersonParser...
❌ Fehler: 'PersonParser' object has no attribute 'parse_batch'


## ActivityParser - Aktivitäten analysieren

In [6]:
# Aktivitäten laden und parsen
print("🎯 Teste ActivityParser...")
try:
    activities = dip.get_aktivitaet(anzahl=3)
    if activities:
        parser = ActivityParser()
        batch_activities = parser.parse_batch(activities)
        
        print(f"✅ {len(batch_activities)} Aktivitäten geparst")
        
        # Aktivitäts-Typen analysieren
        from collections import Counter
        types = [a.get('aktivitaetstyp', 'Unbekannt') for a in batch_activities]
        type_counts = Counter(types)
        
        print("📊 Aktivitäts-Typen:")
        for activity_type, count in type_counts.most_common():
            print(f"  - {activity_type}: {count}")
    else:
        print("❌ Keine Aktivitäten gefunden")
except Exception as e:
    print(f"❌ Fehler: {e}")

🎯 Teste ActivityParser...
❌ Fehler: 'ActivityParser' object has no attribute 'parse_batch'


## Zusammenfassung

✅ **Content-Parser erfolgreich getestet!**

### Verfügbare Parser:
- **ProtocolParser**: Plenarprotokolle strukturiert analysieren
- **DocumentParser**: Drucksachen und Dokumente verarbeiten
- **PersonParser**: Abgeordneten-Daten extrahieren
- **ActivityParser**: Aktivitäten und Abstimmungen analysieren

**Weiter mit Notebook 5: Async API! 🚀**